In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df


In [ ]:
train = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')

In [ ]:
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

In [ ]:
# train.drop(['id', 'target'], axis=1).info()

In [ ]:
x_data = train.drop(['id', 'target'], axis=1)
y_data = train.target
x_test = test.drop('id', axis=1)

x_data = x_data.values
x_test = x_test.values


del train, test
gc.collect()

In [ ]:
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

### cnn

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=256)

x_data_image = pca.fit_transform(x_data)
x_test_image = pca.transform(x_test)

In [ ]:
x_data_image = x_data_image.reshape((-1, 16, 16, 1))
x_test_image = x_test_image.reshape((-1, 16, 16, 1))

In [ ]:
del x_data, x_test
gc.collect()

In [ ]:
model = 0
def model_create():
    
    model = models.Sequential()
    model.add(layers.Conv2D(512, (3, 3), activation='relu', input_shape=(16, 16, 1), padding='same'))
    # model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    # model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model 
    
# model.summary()

In [ ]:
# # with tpu_strategy.scope():
# model = models.Sequential()
# model.add(layers.Dense(1000, activation='relu', input_shape=(285,)))
# model.add(layers.Dense(900, activation='relu'))
# model.add(layers.Dense(800, activation='relu'))
# model.add(layers.Dense(700, activation='relu'))
# model.add(layers.Dense(600, activation='relu'))
# model.add(layers.Dense(500, activation='relu'))
# model.add(layers.Dense(100, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))
# model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

n_split = 10
kfold = KFold(n_split)

val_pred = np.zeros(y_data.shape)
y_test = np.zeros((x_test_image.shape[0],))

for i, (train_index, val_index) in enumerate(kfold.split(x_data_image)):
    gc.collect()
    # train model
    print("fold {} training".format(i))
    model = model_create()
    model.fit(x_data_image[train_index], y_data.iloc[train_index], batch_size=512, epochs=5)
    
    # predict val and test
    val_pred[val_index] = model.predict(x_data_image[val_index]).reshape((-1,))
    vla_score = roc_auc_score(y_data.iloc[val_index], val_pred[val_index])
    print("fold {} validation auc score {}".format(i, vla_score))
    
    y_test += model.predict(x_test_image).reshape((-1,)) / n_split
    
# evaluate validation score    
print("val auc score :", roc_auc_score(y_data, val_pred))

In [ ]:
# from sklearn.model_selection import train_test_split
# x_train, x_val, y_train, y_val = train_test_split(x_data_image, y_data, test_size=0.3, random_state=50)

In [ ]:
# print(x_train.shape)

In [ ]:
# model.fit(x_train, y_train, validation_data = (x_val, y_val), batch_size=512, epochs=5)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
# y_val_pred = model.predict(x_val)
# roc_auc_score(y_val, y_val_pred)

In [ ]:
# y_pred = model.predict(x_test_image)

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')
submission.target = y_test
submission.to_csv('submission.csv', index=False)

In [ ]:
import seaborn as sns
sns.histplot(y_test)